In [1]:
import pandas as pd
import json
from tqdm import tqdm
import numpy as np
import random
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, average_precision_score

In [48]:
def parse_json_data(data_json_dir):
    jsonlist = []
    with open(data_json_dir) as file:
        for jsonobj in tqdm(file):
            jsonlist.append(json.loads(jsonobj))
    return jsonlist
def process_json_data(jsonlist):
    #do xg latest jsondata manipulation
    df_list = []
    for i in tqdm(jsonlist):
        transcript_id = list(i.keys())[0]
        transcript_position = list(i[transcript_id].keys())[0]
        five_mer = list(i[transcript_id][transcript_position].keys())[0]
        readings = i[transcript_id][transcript_position][five_mer]
        df = pd.DataFrame(readings)
        df.columns = ['dwell_time_-1','sd_-1','mean_-1','dwell_time_0','sd_0','mean_0','dwell_time_1','sd_1','mean_1']
        df['product_mean_dwell_-1'] = df['dwell_time_-1']*df['mean_-1']
        df['product_mean_dwell_0'] = df['dwell_time_0']*df['mean_0']
        df['product_mean_dwell_1'] = df['dwell_time_1']*df['mean_1']
        df['product_var_dwell_-1'] = df['sd_-1']*df['sd_-1']*(df['dwell_time_-1'])
        df['product_var_dwell_0'] = df['sd_0']*df['sd_0']*(df['dwell_time_0'])
        df['product_var_dwell_1'] = df['sd_1']*df['sd_1']*(df['dwell_time_1'])

        weighted_mean_neg1 = df['product_mean_dwell_-1'].sum()/df['dwell_time_-1'].sum()
        weighted_mean_0 = df['product_mean_dwell_0'].sum()/df['dwell_time_0'].sum()
        weighted_mean_1 = df['product_mean_dwell_1'].sum()/df['dwell_time_1'].sum()

        weighted_sd_neg1 = np.sqrt(df['product_var_dwell_-1'].sum()/df['dwell_time_-1'].sum())
        weighted_sd_0 = np.sqrt(df['product_var_dwell_0'].sum()/df['dwell_time_0'].sum())
        weighted_sd_1 = np.sqrt(df['product_var_dwell_1'].sum()/df['dwell_time_1'].sum())
        
        mean_25_neg1 = df['mean_-1'].quantile(0.25)
        mean_25_0 = df['mean_0'].quantile(0.25)
        mean_25_1 = df['mean_1'].quantile(0.25)

        mean_50_neg1 = df['mean_-1'].quantile(0.5)
        mean_50_0 = df['mean_0'].quantile(0.5)
        mean_50_1 = df['mean_1'].quantile(0.5)

        mean_75_neg1 = df['mean_-1'].quantile(0.75)
        mean_75_0 = df['mean_0'].quantile(0.75)
        mean_75_1 = df['mean_1'].quantile(0.75)

        sd_25_neg1 = df['sd_-1'].quantile(0.25)
        sd_25_0= df['sd_0'].quantile(0.25)
        sd_25_1= df['sd_1'].quantile(0.25)

        sd_50_neg1 = df['sd_-1'].quantile(0.5)
        sd_50_0= df['sd_0'].quantile(0.5)
        sd_50_1= df['sd_1'].quantile(0.5)

        sd_75_neg1 = df['sd_-1'].quantile(0.75)
        sd_75_0 = df['sd_0'].quantile(0.75)
        sd_75_1 = df['sd_1'].quantile(0.75)
        
        df_list.append([transcript_id,transcript_position,five_mer,weighted_mean_neg1,weighted_mean_0,weighted_mean_1,weighted_sd_neg1,
                weighted_sd_0,weighted_sd_1,mean_25_neg1,mean_25_0,mean_25_1,mean_50_neg1,mean_50_0,mean_50_1,mean_75_neg1,
                mean_75_0,mean_75_1,sd_25_neg1,sd_25_0,sd_25_1,sd_50_neg1,sd_50_0,sd_50_1,sd_75_neg1,sd_75_0,sd_75_1])
    df = pd.DataFrame(df_list)
    df.columns = ['transcript_id','transcript_position','five_mer','weighted_mean_neg1','weighted_mean_0','weighted_mean_1','weighted_sd_neg1',
                'weighted_sd_0','weighted_sd_1','mean_25_neg1','mean_25_0','mean_25_1','mean_50_neg1','mean_50_0','mean_50_1','mean_75_neg1',
                'mean_75_0','mean_75_1','sd_25_neg1','sd_25_0','sd_25_1','sd_50_neg1','sd_50_0','sd_50_1','sd_75_neg1','sd_75_0','sd_75_1']
    return df

def xg_generate_features_new(data_df):
    data_df['5-mer-0'] = data_df['five_mer'].map(lambda x:x[0])
    data_df['5-mer-1'] = data_df['five_mer'].map(lambda x:x[1])
    data_df['5-mer-2'] = data_df['five_mer'].map(lambda x:x[2])
    data_df['5-mer-5'] = data_df['five_mer'].map(lambda x:x[5])
    data_df['5-mer-6'] = data_df['five_mer'].map(lambda x:x[6])
    data_df['5-mer_window-1'] = data_df['five_mer'].map(lambda x: x[0:5])
    data_df['5-mer_window0'] = data_df['five_mer'].map(lambda x: x[1:6])
    data_df['5-mer_window1'] = data_df['five_mer'].map(lambda x: x[2:7])
    data_df_cat = data_df[['5-mer-0','5-mer-1','5-mer-2','5-mer-5','5-mer-6','5-mer_window-1', '5-mer_window0', '5-mer_window1']]
    encoder = OneHotEncoder()
    one_hot_encoded = encoder.fit_transform(data_df_cat)
    one_hot_encoded_df = pd.DataFrame(one_hot_encoded.toarray(), columns=encoder.get_feature_names_out(input_features=data_df_cat.columns))
    data_encoded = pd.concat([data_df.drop(columns=data_df_cat.columns), one_hot_encoded_df], axis=1)
    
    data_encoded['A_freq'] = data_encoded['five_mer'].map(lambda x:x.count('A'))
    data_encoded['C_freq'] = data_encoded['five_mer'].map(lambda x:x.count('C'))
    data_encoded['G_freq'] = data_encoded['five_mer'].map(lambda x:x.count('G'))
    data_encoded['T_freq'] = data_encoded['five_mer'].map(lambda x:x.count('T'))
    return data_encoded

def prediction_pipeline(test_df, model, scaler):
    restricted_columns = ['transcript_id', 'transcript_position','five_mer']
    x_test = test_df[[i for i in test_df.columns if i not in restricted_columns]]
    x_test_scaled = scaler.transform(x_test)
    test_df['score'] = model.predict(x_test_scaled)
    return test_df
    

In [6]:
datasets = ['dataset0','dataset1','dataset2']

In [7]:
json_list_dict = {}
for i in datasets:
    json_list_dict[i] = parse_json_data(f"{i}.json")

121838it [00:43, 2784.31it/s]
90810it [01:05, 1396.83it/s]
1323it [00:03, 343.34it/s]


In [17]:
df_dict = {}
for i in json_list_dict:
    df_dict[i] = process_json_data(json_list_dict[i])

  0%|          | 0/121838 [00:00<?, ?it/s]

100%|██████████| 1323/1323 [00:22<00:00, 58.79it/s]


In [44]:
df_dict_final = {}
for i in df_dict:
    df_dict_final[i] = xg_generate_features_new(df_dict[i])

In [46]:
df_dict_final['dataset0']

,transcript_id,transcript_position,five_mer,weighted_mean_neg1,weighted_mean_0,weighted_mean_1,weighted_sd_neg1,weighted_sd_0,weighted_sd_1,mean_25_neg1,...,5-mer_window1_GACCG,5-mer_window1_GACCT,5-mer_window1_GACTA,5-mer_window1_GACTC,5-mer_window1_GACTG,5-mer_window1_GACTT,A_freq,C_freq,G_freq,T_freq
0,ENST00000000233,244,AAGACCA,123.762870,125.793483,80.775369,5.209304,8.511144,5.686085,123.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4,2,1,0
1,ENST00000000233,261,CAAACTG,109.924484,108.101783,94.108586,3.910096,3.735377,3.442957,108.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3,2,1,1
2,ENST00000000233,316,GAAACAG,105.450998,99.426169,89.309704,3.400684,3.910397,2.501662,105.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4,1,2,0
3,ENST00000000233,332,AGAACAT,129.548782,97.842815,89.096953,6.542432,3.132313,2.372598,128.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4,1,1,1
4,ENST00000000233,368,AGGACAA,118.217577,121.925694,84.996204,7.490865,6.224289,4.516338,116.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,ENST00000641834,1348,GGGACAT,117.736860,116.644041,81.604529,3.679951,5.318444,4.342628,117.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2,1,3,1
121834,ENST00000641834,1429,CTGACAC,111.417384,115.248207,80.183728,5.835194,9.597531,3.848341,110.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2,3,1,1
121835,ENST00000641834,1531,TGGACAC,113.627173,113.706528,84.184182,4.313860,5.075660,2.339143,113.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2,2,2,1
121836,ENST00000641834,1537,CTGACCA,109.836890,123.712610,82.354740,3.270741,7.163837,2.847985,108.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2,3,1,1


In [30]:
for i in df_dict_final:
    df_dict_final[i].to_csv(f'{i}_processed.csv',index = False)

In [31]:
import keras
import tensorflow as tf
import pickle
model = keras.models.load_model('../models/fitted_kerasmlp.h5')
file = open('../models/fitted_scaler.pkl','rb')
scaler = pickle.load(file)
file.close()

c:\Python39\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [49]:
for i in df_dict_final:
    df_dict_final[i] = prediction_pipeline(df_dict_final[i], model, scaler)

42/42 [==============================] - 0s 1ms/step


In [50]:
df_dict_final['dataset0']

,transcript_id,transcript_position,five_mer,weighted_mean_neg1,weighted_mean_0,weighted_mean_1,weighted_sd_neg1,weighted_sd_0,weighted_sd_1,mean_25_neg1,...,5-mer_window1_GACCT,5-mer_window1_GACTA,5-mer_window1_GACTC,5-mer_window1_GACTG,5-mer_window1_GACTT,A_freq,C_freq,G_freq,T_freq,score
0,ENST00000000233,244,AAGACCA,123.762870,125.793483,80.775369,5.209304,8.511144,5.686085,123.0,...,0.0,0.0,0.0,0.0,0.0,4,2,1,0,0.094697
1,ENST00000000233,261,CAAACTG,109.924484,108.101783,94.108586,3.910096,3.735377,3.442957,108.0,...,0.0,0.0,0.0,0.0,0.0,3,2,1,1,0.211056
2,ENST00000000233,316,GAAACAG,105.450998,99.426169,89.309704,3.400684,3.910397,2.501662,105.0,...,0.0,0.0,0.0,0.0,0.0,4,1,2,0,0.008409
3,ENST00000000233,332,AGAACAT,129.548782,97.842815,89.096953,6.542432,3.132313,2.372598,128.0,...,0.0,0.0,0.0,0.0,0.0,4,1,1,1,0.636733
4,ENST00000000233,368,AGGACAA,118.217577,121.925694,84.996204,7.490865,6.224289,4.516338,116.0,...,0.0,0.0,0.0,0.0,0.0,4,1,2,0,0.244850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,ENST00000641834,1348,GGGACAT,117.736860,116.644041,81.604529,3.679951,5.318444,4.342628,117.0,...,0.0,0.0,0.0,0.0,0.0,2,1,3,1,0.976419
121834,ENST00000641834,1429,CTGACAC,111.417384,115.248207,80.183728,5.835194,9.597531,3.848341,110.0,...,0.0,0.0,0.0,0.0,0.0,2,3,1,1,0.029714
121835,ENST00000641834,1531,TGGACAC,113.627173,113.706528,84.184182,4.313860,5.075660,2.339143,113.0,...,0.0,0.0,0.0,0.0,0.0,2,2,2,1,0.978516
121836,ENST00000641834,1537,CTGACCA,109.836890,123.712610,82.354740,3.270741,7.163837,2.847985,108.0,...,0.0,0.0,0.0,0.0,0.0,2,3,1,1,0.084789


In [52]:
for i in df_dict_final:
    new_df = df_dict_final[i][['transcript_id','transcript_position','score']]
    new_df.to_csv(f'{i}_predictions.csv', index = False)

In [57]:
#Sanity Check
dataset0_labels = pd.read_csv('../data/data.info')
dataset0_predict = pd.read_csv('dataset0_predictions.csv')
dataset0_merged = pd.merge(dataset0_labels,dataset0_predict, on = ['transcript_id','transcript_position'])

In [63]:
y_true = dataset0_merged['label']
y_pred = dataset0_merged['score']

In [64]:
roc_auc_score(y_true, y_pred)

0.9230017049857796

In [65]:
average_precision_score(y_true, y_pred)

0.4770497861626484